# Using file search tool in the Responses API

Although RAG can be overwhelming, searching amongst PDF file shouldn't be complicated. One of the most adopted options as of now is parsing your PDF, defining your chunking strategies, uploading those chunks to a storage provider, running embeddings on those chunks of texts and storing those embeddings in a vector database. And that's only the setup — retrieving content in our LLM workflow also requires multiple steps.

This is where file search — a hosted tool you can use in the Responses API — comes in. It allows you to search your knowledge base and generate an answer based on the retrieved content. In this cookbook, we'll upload those PDFs to a vector store on OpenAI and use file search to fetch additional context from this vector store to answer the questions we generated in the first step. Then, we'll initially create a small set of questions based on PDFs extracted from OpenAI's blog ([openai.com/news](https://openai.com/news)).

_File search was previously available on the Assistants API. It's now available on the new Responses API, an API that can be stateful or stateless, and with from new features like metadata filtering_

### Set up

In [22]:
!pip install PyPDF2 pandas tqdm sentence-transformers faiss-cpu transformers torch -q



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 872.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.8 MB/s eta 0:00:00


In [23]:
! pip install python-dotenv

In [25]:
from dotenv import load_dotenv
import os
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import PyPDF2
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from transformers import pipeline
import torch




In [48]:
load_dotenv()
dir_pdfs = 'openai_blog_pdfs' # have those PDFs stored locally here
if not os.path.exists(dir_pdfs):
  print(f"Directory '{dir_pdfs}' not found. Please create it and place your PDF files inside.")

else:
  pdf_files = [os.path.join(dir_pdfs, f) for f in os.listdir(dir_pdfs) if f.endswith('.pdf')]
if not pdf_files:
  print(f"No PDF files found in '{dir_pdfs}'. Please add some PDF files to the directory.")



In [49]:
def extract_text_from_pdf(pdf_path):
   """Extracts text from a single PDF file."""
   text = ""
   try:
    with open(pdf_path, "rb") as f:
      reader = PyPDF2.PdfReader(f)
      for page in reader.pages:
        page_text = page.extract_text()
        if page_text:

                # Simple chunking strategy: treat each page as a chunk
                # You might need a more sophisticated chunking method for better results
                text += page_text + "\n----PAGE_BREAK----\n" # Add a marker to potentially split later
   except Exception as e:
    print(f"Error reading {pdf_path}: {e}")
   return text

In [50]:
def chunk_text(text, chunk_size=500, chunk_overlap=50):
  """Splits text into smaller chunks with overlap."""
  chunks = []

  # A more robust chunking strategy would be beneficial here
  # For simplicity, we'll split by page break and then by a fixed size if pages are too large
  pages = text.split("\n----PAGE_BREAK----\n")
  for page in pages:
    if len(page) > chunk_size:
        words = page.split()
        for i in range(0, len(words), chunk_size - chunk_overlap):
            chunk = " ".join(words[i:i + chunk_size])
            if chunk:
                chunks.append(chunk)
    elif page.strip():
        chunks.append(page.strip())
  return chunks

In [51]:
def create_embeddings(chunks, model):
   """Generates embeddings for text chunks using a SentenceTransformer model."""
   print(f"Creating embeddings for {len(chunks)} chunks...")
   embeddings = model.encode(chunks, show_progress_bar=True)
   return embeddings

In [53]:
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
embedding_dimension = embedding_model.get_sentence_embedding_dimension()

all_chunks = []
chunk_metadata = [] # To store info like source file for each chunk

print(f"Processing {len(pdf_files)} PDF files...")
for pdf_file in tqdm(pdf_files):
  text = extract_text_from_pdf(pdf_file)
  chunks = chunk_text(text)
  all_chunks.extend(chunks)
  for i, chunk in enumerate(chunks):
    chunk_metadata.append({"source": os.path.basename(pdf_file), "chunk_index": i})

Processing 1 PDF files...


100%|██████████| 1/1 [00:01<00:00,  1.25s/it]


In [54]:
chunk_embeddings = create_embeddings(all_chunks, embedding_model)
index = faiss.IndexFlatL2(embedding_dimension)
index.add(np.array(chunk_embeddings))

print(f"FAISS index created with {index.ntotal} embeddings.")


Creating embeddings for 92 chunks...


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

FAISS index created with 92 embeddings.


In [55]:
def retrieve_chunks(query: str, index, embedding_model, all_chunks, chunk_metadata, k=5):
 """Retrieves top-k most similar chunks for a given query."""
 # Create embedding for the query
 query_embedding = embedding_model.encode(query)
 query_embedding = np.array([query_embedding]) # Reshape for FAISS search

 # Search the FAISS index
 distances, indices = index.search(query_embedding, k)

# Get the retrieved chunks and their metadata
 retrieved = []
 for i in indices[0]:
    if i != -1: # Check if a valid index was returned
         retrieved.append({
             "text": all_chunks[i],
             "metadata": chunk_metadata[i],
             "distance": distances[0][list(indices[0]).index(i)]
         })
 return retrieved

try:
  generator = pipeline('text-generation', model='distilgpt2')
  print("Text generation pipeline loaded using distilgpt2.")
except Exception as e:
  print(f"Error loading text generation pipeline: {e}")
  print("You might need to choose a smaller model or run this on a machine with more resources.")
  generator = None # Set to None if loading fails


Device set to use cpu


Text generation pipeline loaded using distilgpt2.


In [56]:
def generate_answer_with_rag(query: str, retrieved_chunks, generator):
   """Generates an answer using a free LLM based on retrieved chunks."""
   if not generator:
    return "Language model not loaded. Cannot generate answer."

   context = "\n".join([chunk['text'] for chunk in retrieved_chunks])

   if not context.strip():
      return "No relevant information found in the documents."

    # Craft the prompt for the language model
    # This prompt structure is crucial for getting good results
   prompt = f"""Based on the following information, answer the query:
    Information: {context}

   Query: {query}

   Answer: """

   print("Generating answer with LLM...")
    # Generate text. You might need to adjust max_new_tokens and other parameters
   response = generator(prompt, max_new_tokens=200, num_return_sequences=1, do_sample=True)[0]['generated_text']

    # Post-process the response to potentially remove the prompt part
    # This is a simple approach, more sophisticated parsing might be needed
   answer_prefix = "Answer:\n"
   if answer_prefix in response:
        answer = response.split(answer_prefix, 1)[1].strip()
   else:
        answer = response.strip()

   return answer

In [59]:
query = "What are the different types of agent"
print(f"\nRetrieving chunks for query: '{query}'...")
retrieved_chunks = retrieve_chunks(query, index, embedding_model, all_chunks, chunk_metadata, k=3) # Retrieve top 3 chunks

if retrieved_chunks:
  print(f"Found {len(retrieved_chunks)} relevant chunks.")

# Optional: Print retrieved chunks and their source
# for i, chunk in enumerate(retrieved_chunks):
#     print(f"--- Chunk {i+1} (Source: {chunk['metadata']['source']}, Distance: {chunk['distance']:.4f}) ---")
#     print(chunk['text'][:300] + "...") # Print first 300 characters
#     print("-" * 20)

# 2. Generate answer using the LLM and retrieved chunks
  answer = generate_answer_with_rag(query, retrieved_chunks, generator)

  print("\nGenerated Answer:")
  print(answer)
else: print("No relevant chunks found for the query.")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Retrieving chunks for query: 'What are the different types of agent'...
Found 3 relevant chunks.
Generating answer with LLM...

Generated Answer:
Based on the following information, answer the query:
    Information: Contents Types of AI Agents When to Use Agents? When Not to Use Agents? 10 Questions to Ask Before You Consider an AI Agent 3 Interesting Real-World Use Cases of AI Agents10 30 21 31 31 32 33 34 34 35 35 35 37 4022 23 25Chapter 1: What are AI agentsChapter 2: Frameworks for Building Agents LangGraph vs. AutoGen vs. CrewAI Practical Considerations What Tools and Functionalities Do They Support? How Well Do They Maintain the Context? Are They Well-Organized and Easy to Interpret? What’s the Quality of Documentation? Do They Provide Multi-Agent Support? What About Caching? Looking at the Replay Functionality What About Code Execution? Human in the Loop Support? Popular Use Cases Centered Around These Frameworks7/27 28/43
10 Mastering AI Agents Types of AI Agents Now that we’

In [16]:
from openai import OpenAI
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import concurrent
import PyPDF2
import os
import pandas as pd
import base64

client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
dir_pdfs = 'openai_blog_pdfs' # have those PDFs stored locally here
pdf_files = [os.path.join(dir_pdfs, f) for f in os.listdir(dir_pdfs)]

### Creating Vector Store with our PDFs

We will create a Vector Store on OpenAI API and upload our PDFs to the Vector Store. OpenAI will read those PDFs, separate the content into multiple chunks of text, run embeddings on those and store those embeddings and the text in the Vector Store. It will enable us to query this Vector Store to return relevant content based on a query.

In [17]:
def upload_single_pdf(file_path: str, vector_store_id: str):
    file_name = os.path.basename(file_path)
    try:
        file_response = client.files.create(file=open(file_path, 'rb'), purpose="assistants")
        attach_response = client.vector_stores.files.create(
            vector_store_id=vector_store_id,
            file_id=file_response.id
        )
        return {"file": file_name, "status": "success"}
    except Exception as e:
        print(f"Error with {file_name}: {str(e)}")
        return {"file": file_name, "status": "failed", "error": str(e)}

def upload_pdf_files_to_vector_store(vector_store_id: str):
    pdf_files = [os.path.join(dir_pdfs, f) for f in os.listdir(dir_pdfs)]
    stats = {"total_files": len(pdf_files), "successful_uploads": 0, "failed_uploads": 0, "errors": []}

    print(f"{len(pdf_files)} PDF files to process. Uploading in parallel...")

    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(upload_single_pdf, file_path, vector_store_id): file_path for file_path in pdf_files}
        for future in tqdm(concurrent.futures.as_completed(futures), total=len(pdf_files)):
            result = future.result()
            if result["status"] == "success":
                stats["successful_uploads"] += 1
            else:
                stats["failed_uploads"] += 1
                stats["errors"].append(result)

    return stats

def create_vector_store(store_name: str) -> dict:
    try:
        vector_store = client.vector_stores.create(name=store_name)
        details = {
            "id": vector_store.id,
            "name": vector_store.name,
            "created_at": vector_store.created_at,
            "file_count": vector_store.file_counts.completed
        }
        print("Vector store created:", details)
        return details
    except Exception as e:
        print(f"Error creating vector store: {e}")
        return {}

In [18]:
store_name = "ai_agents_store"
vector_store_details = create_vector_store(store_name)
upload_pdf_files_to_vector_store(vector_store_details["id"])

Vector store created: {'id': 'vs_6857dd3e1dec8191b1de5c533c4748cb', 'name': 'ai_agents_store', 'created_at': 1750588734, 'file_count': 0}
2 PDF files to process. Uploading in parallel...


100%|██████████| 2/2 [00:08<00:00,  4.23s/it]


{'total_files': 2, 'successful_uploads': 2, 'failed_uploads': 0, 'errors': []}

### Standalone vector search

Now that our vector store is ready, we are able to query the Vector Store directly and retrieve relevant content for a specific query. Using the new [vector search API](https://platform.openai.com/docs/api-reference/vector-stores/search), we're able to find relevant items from our knowledge base without necessarily integrating it in an LLM query.

In [19]:
query = "What's AI agents"
search_results = client.vector_stores.search(
    vector_store_id=vector_store_details['id'],
    query=query
)

In [20]:
for result in search_results.data:
    print(str(len(result.content[0].text)) + ' of character of content from ' + result.filename + ' with a relevant score of ' + str(result.score))

3582 of character of content from Mastering AI agents.pdf with a relevant score of 0.9482921985607468
3515 of character of content from Mastering AI agents.pdf with a relevant score of 0.9348534074942466
3145 of character of content from Newwhitepaper_Agents.pdf with a relevant score of 0.886270160325278
3856 of character of content from Mastering AI agents.pdf with a relevant score of 0.8826236858030915
3373 of character of content from Mastering AI agents.pdf with a relevant score of 0.8796075510298714
3084 of character of content from Mastering AI agents.pdf with a relevant score of 0.8795344108551211
3847 of character of content from Mastering AI agents.pdf with a relevant score of 0.87755298161713
3681 of character of content from Mastering AI agents.pdf with a relevant score of 0.8752338024110329
3897 of character of content from Mastering AI agents.pdf with a relevant score of 0.8689723378176969
3700 of character of content from Mastering AI agents.pdf with a relevant score of 0

We can see that different size (and under-the-hood different texts) have been returned from the search query. They all have different relevancy score that are calculated by our ranker which uses hybrid search.

### Integrating search results with LLM in a single API call

However instead of querying the vector store and then passing the data into the Responses or Chat Completion API call, an even more convenient way to use this search results in an LLM query would be to plug use file_search tool as part of OpenAI Responses API.

In [21]:
query = "What's AI agents"
response = client.responses.create(
    input= query,
    model="gpt-4o-mini",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_details['id']],
    }]
)

# Extract annotations from the response
annotations = response.output[1].content[0].annotations

# Get top-k retrieved filenames
retrieved_files = set([result.filename for result in annotations])

print(f'Files used: {retrieved_files}')
print('Response:')
print(response.output[1].content[0].text) # 0 being the filesearch call

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

We can see that `gpt-4o-mini` was able to answer a query that required more recent, specialised knowledge about OpenAI's Deep Research. It used content from the file `Introducing deep research _ OpenAI.pdf` that had chunks of texts that were the most relevant. If we want to go even deeper in the analysis of chunk of text retrieved, we can also analyse the different texts that were returned by the search engine by adding `include=["output[*].file_search_call.search_results"]` to our query.

## Evaluating performance

What is key for those information retrieval system is to also measure the relevance & quality of files retrieved for those answers. The following steps of this cookbook will consist in generating an evaluation dataset and calculating different metrics over this generated dataset. This is an imperfect approach and we'll always recommend to have a human-verified evaluation dataset for your own use-cases, but it will show you the methodology to evaluate those.  It will be imperfect because some of the questions generated might be generic (e.g: What's said by the main stakeholder in this document) and our retrieval test will have a hard time to figure out which document that question was generated for.

### Generating questions

We will create functions that will read through the PDFs we have locally and generate a question that can only be answered by this document. Therefore it'll create our evaluation dataset that we can use after.

In [ ]:
def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with open(pdf_path, "rb") as f:
            reader = PyPDF2.PdfReader(f)
            for page in reader.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text
    except Exception as e:
        print(f"Error reading {pdf_path}: {e}")
    return text

def generate_questions(pdf_path):
    text = extract_text_from_pdf(pdf_path)

    prompt = (
        "Can you generate a question that can only be answered from this document?:\n"
        f"{text}\n\n"
    )

    response = client.responses.create(
        input=prompt,
        model="gpt-4o",
    )

    question = response.output[0].content[0].text

    return question

If we run the function generate_question for the first PDF file we will be able to see the kind of question it generates.

In [ ]:
generate_questions(pdf_files[0])

'What new capabilities will ChatGPT have as a result of the partnership between OpenAI and Schibsted Media Group?'

We can now generate all the questions for all the PDFs we've got stored locally.

In [ ]:
# Generate questions for each PDF and store in a dictionary
questions_dict = {}
for pdf_path in pdf_files:
    questions = generate_questions(pdf_path)
    questions_dict[os.path.basename(pdf_path)] = questions

In [ ]:
questions_dict

{'OpenAI partners with Schibsted Media Group _ OpenAI.pdf': 'What is the purpose of the partnership between Schibsted Media Group and OpenAI announced on February 10, 2025?',
 'OpenAI and the CSU system bring AI to 500,000 students & faculty _ OpenAI.pdf': 'What significant milestone did the California State University system achieve by partnering with OpenAI, making it the first of its kind in the United States?',
 '1,000 Scientist AI Jam Session _ OpenAI.pdf': 'What was the specific AI model used during the "1,000 Scientist AI Jam Session" event across the nine national labs?',
 'Announcing The Stargate Project _ OpenAI.pdf': 'What are the initial equity funders and lead partners in The Stargate Project announced by OpenAI, and who holds the financial and operational responsibilities?',
 'Introducing Operator _ OpenAI.pdf': 'What is the name of the new model that powers the Operator agent introduced by OpenAI?',
 'Introducing NextGenAI _ OpenAI.pdf': 'What major initiative did OpenAI

We now have a dictionary of `filename:question` that we can loop through and ask gpt-4o(-mini) about without providing the document, and gpt-4o should be able to find the relevant document in the Vector Store.

We'll convert our dictionary into a dataframe and process it using gpt-4o-mini. We will look out for the expected file

In [ ]:
rows = []
for filename, query in questions_dict.items():
    rows.append({"query": query, "_id": filename.replace(".pdf", "")})

# Metrics evaluation parameters
k = 5
total_queries = len(rows)
correct_retrievals_at_k = 0
reciprocal_ranks = []
average_precisions = []

def process_query(row):
    query = row['query']
    expected_filename = row['_id'] + '.pdf'
    # Call file_search via Responses API
    response = client.responses.create(
        input=query,
        model="gpt-4o-mini",
        tools=[{
            "type": "file_search",
            "vector_store_ids": [vector_store_details['id']],
            "max_num_results": k,
        }],
        tool_choice="required" # it will force the file_search, while not necessary, it's better to enforce it as this is what we're testing
    )
    # Extract annotations from the response
    annotations = None
    if hasattr(response.output[1], 'content') and response.output[1].content:
        annotations = response.output[1].content[0].annotations
    elif hasattr(response.output[1], 'annotations'):
        annotations = response.output[1].annotations

    if annotations is None:
        print(f"No annotations for query: {query}")
        return False, 0, 0

    # Get top-k retrieved filenames
    retrieved_files = [result.filename for result in annotations[:k]]
    if expected_filename in retrieved_files:
        rank = retrieved_files.index(expected_filename) + 1
        rr = 1 / rank
        correct = True
    else:
        rr = 0
        correct = False

    # Calculate Average Precision
    precisions = []
    num_relevant = 0
    for i, fname in enumerate(retrieved_files):
        if fname == expected_filename:
            num_relevant += 1
            precisions.append(num_relevant / (i + 1))
    avg_precision = sum(precisions) / len(precisions) if precisions else 0

    if expected_filename not in retrieved_files:
        print("Expected file NOT found in the retrieved files!")

    if retrieved_files and retrieved_files[0] != expected_filename:
        print(f"Query: {query}")
        print(f"Expected file: {expected_filename}")
        print(f"First retrieved file: {retrieved_files[0]}")
        print(f"Retrieved files: {retrieved_files}")
        print("-" * 50)


    return correct, rr, avg_precision

In [ ]:
process_query(rows[0])

(True, 1.0, 1.0)

Recall & Precision are at 1 for this example, and our file ranked first so we're having a MRR and MAP = 1 on this example.

We can now execute this processing on our set of questions.

In [ ]:
with ThreadPoolExecutor() as executor:
    results = list(tqdm(executor.map(process_query, rows), total=total_queries))

correct_retrievals_at_k = 0
reciprocal_ranks = []
average_precisions = []

for correct, rr, avg_precision in results:
    if correct:
        correct_retrievals_at_k += 1
    reciprocal_ranks.append(rr)
    average_precisions.append(avg_precision)

recall_at_k = correct_retrievals_at_k / total_queries
precision_at_k = recall_at_k  # In this context, same as recall
mrr = sum(reciprocal_ranks) / total_queries
map_score = sum(average_precisions) / total_queries

 62%|███████████████████▏           | 13/21 [00:07<00:03,  2.57it/s]

Expected file NOT found in the retrieved files!
Query: What is OpenAI's mission as stated in the document?
Expected file: Disrupting malicious uses of AI _ OpenAI.pdf
First retrieved file: Introducing the Intelligence Age _ OpenAI.pdf
Retrieved files: ['Introducing the Intelligence Age _ OpenAI.pdf']
--------------------------------------------------


 71%|██████████████████████▏        | 15/21 [00:14<00:06,  1.04s/it]

Expected file NOT found in the retrieved files!
Query: What is the purpose of the "Model Spec" document published by OpenAI for ChatGPT?
Expected file: The power of personalized AI _ OpenAI.pdf
First retrieved file: Sharing the latest Model Spec _ OpenAI.pdf
Retrieved files: ['Sharing the latest Model Spec _ OpenAI.pdf', 'Sharing the latest Model Spec _ OpenAI.pdf', 'Sharing the latest Model Spec _ OpenAI.pdf', 'Sharing the latest Model Spec _ OpenAI.pdf', 'Sharing the latest Model Spec _ OpenAI.pdf']
--------------------------------------------------


100%|███████████████████████████████| 21/21 [00:15<00:00,  1.38it/s]


The outputs logged above would either show that a file wasn't ranked first when our evaluation dataset expected it to rank first or that it wasn't found at all. As we can see from our imperfect evaluation dataset, some questions were generic and expected another doc, which our retrieval system didn't specifically retrieved for this question.

In [ ]:
# Print the metrics with k
print(f"Metrics at k={k}:")
print(f"Recall@{k}: {recall_at_k:.4f}")
print(f"Precision@{k}: {precision_at_k:.4f}")
print(f"Mean Reciprocal Rank (MRR): {mrr:.4f}")
print(f"Mean Average Precision (MAP): {map_score:.4f}")

Metrics at k=5:
Recall@5: 0.9048
Precision@5: 0.9048
Mean Reciprocal Rank (MRR): 0.9048
Mean Average Precision (MAP): 0.8954


With this cookbook we were able to see how to:
- Generate a dataset of evaluations using PDF context-stuffing (leveraging vision modality of 4o) and traditional PDF readers
- Create a vector store and populate it with PDF
- Get an LLM answer to a query, leveraging a RAG system available out-of-the-box with `file_search` tool call in OpenAI's Response API
- Understand how chunks of texts are retrieved, ranked and used as part of the Response API
- Measure accuracy, precision, retrieval, MRR and MAP on the dataset of evaluations previously generated

By using file search with Responses, you can simplify RAG architecture and leverage this in a single API call using the new Responses API. File storage, embeddings, retrieval all integrated in one tool!
